In [52]:
#%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [53]:
transform1 = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform1)
trainloader= torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
train_size = int(0.5 * len(trainset))
trainset1, trainset2 = torch.utils.data.random_split(trainset, [train_size, train_size])

trainloader1= torch.utils.data.DataLoader(trainset1, batch_size=4,
                                          shuffle=True, num_workers=2)
trainloader2 = torch.utils.data.DataLoader(trainset2, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform1)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
print(len(trainset))
print(len(trainset1))
print(len(trainset2))
print(len(testset))


classes = ('beaver', 'dolphin', 'otter', 'seal', 'whale', 'orchids', 'poppies', 'roses',
 'sunflowers', 'tulips', 'aquarium_fish', 'flatfish', 'ray', 'shark', 'trout', 'bottles'
 , 'bowls', 'cans', 'cups', 'plates', 'clock', 'computer_keyboard', 'lamp', 'telephone'
 , 'television', 'apples', 'mushrooms', 'oranges', 'pears', 'sweet_peppers',
 'bed', 'chair', 'couch', 'table', 'wardrobe', 'bear', 'leopard', 'lion', 'tiger', 'wolf'
 , 'bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach', 'bridge', 'castle', 'house',
  'road', 'skyscraper', 'cloud', 'forest', 'mountain', 'plain', 'sea', 'fox', 'porcupine'
  , 'possum', 'raccoon', 'skunk','camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo', 
  'crab', 'lobster', 'snail', 'spider', 'worm', 'crocodile', 'dinosaur', 'lizard', 'snake'
  , 'turtle', 'baby', 'boy', 'girl', 'man', 'woman', 'maple', 'oak', 'palm', 'pine'
  , 'willow', 'hamster', 'mouse', 'rabbit', 'shrew', 'squirrel', 'bicycle', 'bus', 
  'motorcycle', 'pickup_truck', 'train', 'lawn-mower', 'rocket', 'streetcar', 'tank', 'tractor')

Files already downloaded and verified
Files already downloaded and verified
50000
25000
25000
10000


In [54]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(classes))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [55]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [56]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader1, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training train set 1')

[1,  2000] loss: 6.605
[1,  4000] loss: 4.669
[1,  6000] loss: 4.387
[2,  2000] loss: 4.166
[2,  4000] loss: 4.019
[2,  6000] loss: 3.932
Finished Training train set 1


In [57]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy1= float(correct)/float(total)*100
print('Accuracy of the network on the 10000 test images: %.2f ' % accuracy1)

Accuracy of the network on the 10000 test images: 9.20 


In [58]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
correct_indicator = [0 for i in range(len(testset))]
i = 0
# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            
            if label == prediction:
                correct_pred[classes[label]] += 1
                correct_indicator[i] = 1
            total_pred[classes[label]] += 1
            i += 1

# 1 - 1, 2 - 1, 3 - 1, 4-0, 5-0, 6-0 - 60
# 2, 3, 4, 5 - 80
print(correct_indicator[0:20])

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
Accuracy for class: beaver is 0.0 %
Accuracy for class: dolphin is 9.0 %
Accuracy for class: otter is 15.0 %
Accuracy for class: seal  is 18.0 %
Accuracy for class: whale is 0.0 %
Accuracy for class: orchids is 6.0 %
Accuracy for class: poppies is 8.0 %
Accuracy for class: roses is 0.0 %
Accuracy for class: sunflowers is 0.0 %
Accuracy for class: tulips is 16.0 %
Accuracy for class: aquarium_fish is 0.0 %
Accuracy for class: flatfish is 4.0 %
Accuracy for class: ray   is 0.0 %
Accuracy for class: shark is 0.0 %
Accuracy for class: trout is 0.0 %
Accuracy for class: bottles is 3.0 %
Accuracy for class: bowls is 0.0 %
Accuracy for class: cans  is 65.0 %
Accuracy for class: cups  is 0.0 %
Accuracy for class: plates is 0.0 %
Accuracy for class: clock is 0.0 %
Accuracy for class: computer_keyboard is 0.0 %
Accuracy for class: lamp  is 3.0 %
Accuracy for class: telephone is 0.0 %
Accuracy for class: television is 52.0 %
Accuracy fo

In [59]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader2, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training train set 2')

[1,  2000] loss: 3.844
[1,  4000] loss: 3.747
[1,  6000] loss: 3.692
[2,  2000] loss: 3.588
[2,  4000] loss: 3.569
[2,  6000] loss: 3.519
Finished Training train set 2


In [60]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy2= float(correct)/float(total)*100


print('Accuracy of the network on the 10000 test images: %.2f ' % accuracy2)

Accuracy of the network on the 10000 test images: 16.84 


In [61]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
correct_indicator = [0 for i in range(len(testset))]
i = 0
# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            
            if label == prediction:
                correct_pred[classes[label]] += 1
                correct_indicator[i] = 1
            total_pred[classes[label]] += 1
            i += 1

# 1 - 1, 2 - 1, 3 - 1, 4-0, 5-0, 6-0 - 60
# 2, 3, 4, 5 - 80
print(correct_indicator[0:20])

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1]
Accuracy for class: beaver is 59.0 %
Accuracy for class: dolphin is 19.0 %
Accuracy for class: otter is 34.0 %
Accuracy for class: seal  is 10.0 %
Accuracy for class: whale is 0.0 %
Accuracy for class: orchids is 11.0 %
Accuracy for class: poppies is 27.0 %
Accuracy for class: roses is 23.0 %
Accuracy for class: sunflowers is 19.0 %
Accuracy for class: tulips is 19.0 %
Accuracy for class: aquarium_fish is 14.0 %
Accuracy for class: flatfish is 5.0 %
Accuracy for class: ray   is 31.0 %
Accuracy for class: shark is 24.0 %
Accuracy for class: trout is 4.0 %
Accuracy for class: bottles is 5.0 %
Accuracy for class: bowls is 6.0 %
Accuracy for class: cans  is 64.0 %
Accuracy for class: cups  is 30.0 %
Accuracy for class: plates is 5.0 %
Accuracy for class: clock is 17.0 %
Accuracy for class: computer_keyboard is 1.0 %
Accuracy for class: lamp  is 9.0 %
Accuracy for class: telephone is 27.0 %
Accuracy for class: television is 28.0 %

In [63]:
forgetfullness = (accuracy1-accuracy2)
print('forgetting percentage of the network is: %f %%' % forgetfullness)

forgetting percentage of the network is: -7.640000 %
